# Contextual Bandits data

## Load data

In [ ]:
# Initialize Workspace
from azureml.core import Workspace
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
# Create or Attach AML Compute
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "rlos-cfe-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

In [ ]:
import shutil, os
source_directory = 'script-run'
shutil.rmtree(source_directory)
os.makedirs(source_directory, exist_ok=True)
shutil.copytree('cb', os.path.join(source_directory, 'cb'))

In [ ]:
%%writefile $source_directory/setup_mpi.py
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
import sys
import os
from mpi4py import MPI
import pandas as pd
from cb.estimate import estimate

mounted_input_path = sys.argv[1]

print("Argument 1: %s" % mounted_input_path)
print("Number of nodes {}:".format(MPI.COMM_WORLD.Get_size()))
print("Node index {}:".format(MPI.COMM_WORLD.Get_rank()))

even = []
odd = [] 

input_file_list = os.listdir(mounted_input_path)
print("File number: {}".format(len(input_file_list)))
for index in range(0, len(input_file_list)):
    if index % 2 == 0:
        even.append(mounted_input_path + "/" + input_file_list[index])
        print("read in {} file".format(input_file_list[index]))
    else:
        odd.append(mounted_input_path + "/" + input_file_list[index])
        print("read in {} file".format(input_file_list[index]))
print(even, odd)
 
config = {
   'policies': {'random': ['ips', 'snips'], 'baseline1': ['ips']},
   'aggregation': {'num_of_events': 10}
}

input_files = []
if MPI.COMM_WORLD.Get_rank() == 0:
    input_files = odd
elif MPI.COMM_WORLD.Get_rank() == 1:
    input_files = even
mounted_output_path = "outputs"
os.makedirs(mounted_output_path, exist_ok=True)
estimate(input_files, config, mounted_output_path)

In [ ]:
def_blob_store = ws.get_default_datastore()
print(def_blob_store)
import sys
for p in sys.path:
    print(p)
print(sys.path)
def_blob_store.upload_files(files = ["test_data/cb/01.csv", "test_data/cb/02.csv"],
                        target_path = '/dataset/cb/',
                        overwrite = True,
                        show_progress = True)

In [ ]:
from azureml.core import Dataset
from azureml.data import OutputFileDatasetConfig

input_data = Dataset.File.from_files(def_blob_store.path('/dataset/cb/*')).as_named_input('input').as_mount()

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DockerConfiguration
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment("myenv")

myenv.docker.enabled = True
myenv.python.conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk>=1.12.0', 'pandas','matplotlib','mpi4py','vw-estimators'])
myenv.docker.base_image = 'mcr.microsoft.com/azureml/intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04'
docker_config = DockerConfiguration(use_docker=True)

In [ ]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import MpiConfiguration

src = ScriptRunConfig(source_directory=source_directory, 
                      script='setup_mpi.py', 
                      # to mount the dataset on the remote compute and pass the mounted path as an argument to the training script
                      arguments =[input_data],
                      compute_target=compute_target,
                      environment=myenv,
                      distributed_job_config=MpiConfiguration(node_count=2))

In [ ]:
## Build and Submit Experiment
from azureml.core import Experiment
exp = Experiment(ws, 'ScriptRun_parallel')
run = exp.submit(config=src)

In [ ]:
# View Run Details
run.wait_for_completion(show_output=True)

In [ ]:
run.download_files()

## Visualize

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
def plot_estimate(df: pd.DataFrame):
    for index in range(1, len(df.columns)):
        estimate = df[df.columns[index]]
        plt.plot(estimate, "-o", label=df.columns.values[index])

In [ ]:
from estimate import estimate
import os
from os.path import isfile, join

output_files = [os.path.join("outputs", f) for f in sorted(os.listdir("outputs"))]
df = pd.DataFrame()
for file in output_files:
    chunk = pd.read_csv(file)
    df = df.append(chunk, ignore_index = True)
plot_estimate(df)
plt.legend(loc="upper left")
plt.show()